## Definitely DO NOT USE Run All

## importing necessary packages

In [5]:
import numpy as np
import pandas as pd
import pickle
import re
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## reduce memory function

In [6]:
def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

## loading datasets

In [7]:
# games_title_positiveratio = reduce_memory(pd.read_csv("./games.csv",usecols=["app_id","title","positive_ratio"]))
# games_metadata = reduce_memory(pd.read_csv("./games_metadata.csv",usecols=["app_id","description","tags"]))
# games = pd.merge(games_title_positiveratio, games_metadata,on='app_id')
games = reduce_memory(pd.read_csv("./final_games_data_forDesc.csv")) ##or any csv file for recomendation by title 
games.head(5)
# games.to_csv("games_preprocessed_index.csv")

,app_id,title,About the game,Categories,Genres,Tags
0,2525270,The House,' The House ' is a short psychological horror ...,"Single-player,Steam Workshop","Adventure,Casual,Indie","Adventure,Casual,Action-Adventure,Walking Simu..."
1,2414640,Bite Size Terrors: Erobos Heaven,Bite Size Terrors: Are Short Experimental Horr...,"Single-player,Steam Achievements","Adventure,Indie","Horror,Adventure,Atmospheric,Psychological Hor..."
2,2486670,TD Tower Defense,Td Tower Defense is a fun tower defense shoote...,Single-player,"Action,Casual","Tower Defense,Casual,Sci-fi,Strategy,Combat,Re..."
3,2304650,The Scrap,The Scrap is an independent third-person shoot...,"Single-player,Steam Achievements","Action,Adventure,Indie","Action,Shooter,Third-Person Shooter,3D,Third P..."
4,2519670,Wind Love,Gameplay Wind Love - is a Japanese-style visua...,"Single-player,Steam Achievements,Captions avai...","Indie,Simulation","Hentai,Adventure,Dating Sim,Casual,Choices Mat..."


## remove spaces and special character from game name in both dataset

In [10]:
#
for i, row in games.iterrows():
    clean = re.sub('[^A-Za-z0-9]+', ' ', row["title"])
    clean = clean.lower()
    games.at[i, 'ID'] = clean

print(games['ID'].head(8))

0                          the house
1    bite size terrors erobos heaven
2                   td tower defense
3                          the scrap
4                          wind love
5                          greedland
6             bit the apple so what 
7                    nightmare manor
Name: ID, dtype: object


## NLTK

In [ ]:
# import nltk 
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.corpus import wordnet
# from nltk import pos_tag
# from nltk.stem import WordNetLemmatizer

# # Download necessary resources
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger_eng')
  
  
# from nltk.stem import WordNetLemmatizer 
# lemmatizer = WordNetLemmatizer() 
  
# from nltk.corpus import stopwords 
# nltk.download('stopwords') 
# stop_words = set(stopwords.words('english')) 
  
# VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'} 

# def preprocess_sentences(text): 
#   text = text.lower() 
#   temp_sent =[] 
#   words = nltk.word_tokenize(text) 
#   tags = nltk.pos_tag(words) 
#   for i, word in enumerate(words): 
#       if tags[i][1] in VERB_CODES:  
#           lemmatized = lemmatizer.lemmatize(word, 'v') 
#       else: 
#           lemmatized = lemmatizer.lemmatize(word) 
#       if lemmatized not in stop_words and lemmatized.isalpha(): 
#           temp_sent.append(lemmatized) 
          
#   finalsent = ' '.join(temp_sent) 
#   finalsent = finalsent.replace("n't", " not") 
#   finalsent = finalsent.replace("'m", " am") 
#   finalsent = finalsent.replace("'s", " is") 
#   finalsent = finalsent.replace("'re", " are") 
#   finalsent = finalsent.replace("'ll", " will") 
#   finalsent = finalsent.replace("'ve", " have") 
#   finalsent = finalsent.replace("'d", " would") 
#   return finalsent 


# games.dropna(subset=['description'], inplace=True)
# games.set_index('title')

# chunksize = 1000 # Adjust the size based on your memory and performance needs

# # Initialize an empty list to collect processed chunks
# processed_chunks = []

# # Process each chunk
# for chunk in pd.read_csv('games_preprocessed.csv', chunksize=chunksize):  
#     chunk.dropna(subset=['description'], inplace=True)
#     processed_chunk = games["processed_desc"]= games['description'].apply(preprocess_sentences)
#     processed_chunks.append(processed_chunk)

# # Concatenate all processed chunks back into a single DataFrame
# games_processed = pd.concat(processed_chunks, ignore_index=True)
# print(games_processed)

In [1]:
# import spacy
# from spacy.lang.en.stop_words import STOP_WORDS

# # Load SpaCy model
# nlp = spacy.load('en_core_web_sm')

# def preprocess_sentences(text):
#     # Process text with SpaCy
#     doc = nlp(text.lower())
#     # Lemmatize and remove stopwords and non-alphabetic tokens
#     tokens = [token.lemma_ for token in doc if token.text.isalpha() and token.text not in STOP_WORDS]
#     # Join tokens into a final processed string
#     return ' '.join(tokens)

# # Define chunk size
# chunksize = 10000  # Adjust based on your memory and performance needs

# # Initialize an empty list to collect processed chunks
# processed_chunks = []

# # Process each chunk
# for i,chunk in enumerate(pd.read_csv('games_preprocessed.csv', chunksize=chunksize)):
#     chunk.dropna(subset=['description'], inplace=True)
#     # Apply preprocessing to each chunk
#     chunk['processed_desc'] = chunk['description'].apply(preprocess_sentences)
#     processed_chunks.append(chunk)
#     chunk.to_csv(f'processed_chunk_{i}.csv', index=False)

# # Concatenate all processed chunks back into a single DataFrame
# games_processed = pd.concat(processed_chunks, ignore_index=True)

# print(games_processed)


## countvectorizor for title

In [13]:

######################


#not chunking for title
# count = CountVectorizer(stop_words='english')
# games.dropna(subset=['description'], inplace=True)
count = TfidfVectorizer(stop_words='english')
count_matrixx = count.fit_transform(games["ID"])
print(games["ID"])
# count.vocabulary_
feature_names = count.get_feature_names_out()
print(feature_names)
cosine_sim_matrix = cosine_similarity(count_matrixx, dense_output=False)
print(cosine_sim_matrix)
indices = pd.Series(games.index, index=games['title']).drop_duplicates()
print(indices)




# # array = count_matrixx.toarray()
# # df = pd.DataFrame(array, columns=feature_names)
# # print(df)
# # max_loops = 1000

# # for i, row in enumerate(array):
# #     if i >= max_loops:
# #         break  # Stop the loop after 1000 iterations

# #     non_zero_indices = np.nonzero(row)[0]  # Indices of non-zero values
# #     non_zero_features = feature_names[non_zero_indices]  # Corresponding features
# #     non_zero_values = row[non_zero_indices]  # Corresponding counts
    
# #     print(f"Row {i}:")
# #     for feature, value in zip(non_zero_features, non_zero_values):
# #         print(f"  {feature}: {value}")
# #     print("\n" + "-"*50 + "\n")

0                              the house
1        bite size terrors erobos heaven
2                       td tower defense
3                              the scrap
4                              wind love
                      ...               
18555                   learning factory
18556                   fantasy royal vr
18557               buddy simulator 1984
18558                    tower of portal
18559          little elf tower defense 
Name: ID, Length: 18560, dtype: object
['000' '01' '010' ... 'zumba' 'zyconix' 'zympaia']
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1190147 stored elements and shape (18560, 18560)>
  Coords	Values
  (0, 18320)	0.5453676534045102
  (0, 18095)	0.4805285038392838
  (0, 17602)	0.5453676534045102
  (0, 17162)	0.4457120529336221
  (0, 16466)	0.5817895089592608
  (0, 16008)	0.507971971926724
  (0, 14926)	0.616111136197562
  (0, 14904)	0.6548981141824203
  (0, 14564)	0.5006053293921127
  (0, 14068)	0.44388310166125505
  (0, 13448)

## countvectorizor for desc 
##### I am lost what i did here

In [ ]:
# chunk_size = 10000  # Adjust based on memory availability

# chunks = pd.read_csv('processed_chunk_0.csv', chunksize=chunk_size)

# count = TfidfVectorizer(stop_words='english')
# # Initialize an empty list to store chunk results
# chunk_similarities = []

# # Initialize an empty list to store all TF-IDF matrices
# tfidf_matrices = []
# # Initialize a list to store original indices
# original_indices = []

# # Process each chunk (example for combining chunks after processing)
# for chunk in chunks:
#     chunk.dropna(subset=['processed_desc'], inplace=True)
#     indices_chunk = pd.Series(chunk.index, index=chunk['processed_desc'])
#     original_indices.append(indices_chunk)
#     tfidf_matrix_chunk = count.fit_transform(chunk['processed_desc'])
#     tfidf_matrices.append(tfidf_matrix_chunk)

# # Initialize an empty matrix to store cosine similarities
# total_docs = sum(matrix.shape[0] for matrix in tfidf_matrices)
# cosine_sim_matrix = np.zeros((total_docs, total_docs))

# # Initialize row offset
# row_offset = 0

# # Calculate cosine similarities incrementally
# for i, matrix_chunk in enumerate(tfidf_matrices):
#     # Compute cosine similarities for the chunk
#     chunk_similarities.append(cosine_similarity(matrix_chunk, matrix_chunk))
    
#     # Place the similarity results in the appropriate section of the matrix
#     chunk_size = matrix_chunk.shape[0]
#     cosine_sim_matrix[row_offset:row_offset + chunk_size, row_offset:row_offset + chunk_size] = chunk_similarities[-1]
    
#     # Update the row offset
#     row_offset += chunk_size

# print(cosine_sim_matrix)
# all_indices = pd.concat(original_indices)
# # indices = pd.Series(all_indices, index=all_indices).drop_duplicates()
# all_indices.to_csv("desc_indices.csv")
# print(all_indices)

In [ ]:
# n_recommendation = 20
# def get_recommendations(title, cosine_sim):

# 	# if title not in listGames:
#     # 	   return []


# 	desc = games[games['title'] == title]['processed_desc'].values[0]
# 	print(desc)

# 	# Get the index of the game that matches the name
# 	idx = indices[desc]
# 	print(idx)

# 	# if there's 2 games or more with same name (game RUSH)
# 	if type(idx) is pd.Series:
# 		return []
# 	# Get the pairwise similarity scores of all games with that game
# 	sim_scores = list(enumerate(cosine_sim[idx]))

# 	# Sort the games based on the similarity scores
# 	sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# 	# Get the scores of the most similar games
# 	# (not the first one because this games as a score of 1 (perfect score) similarity
#     # with itself)
# 	print(sim_scores)
# 	sim_scores = sim_scores[1:n_recommendation + 1]

# 	# Get the games indices
# 	movie_indices = [i[0] for i in sim_scores]
# 	print(movie_indices)
# 	# Return the top most similar games
# 	return games['title'].iloc[movie_indices].tolist()

# get_recommendations("Prince of Persia®: The Sands of Time", cosine_sim_matrix)
# get_recommendations("Commandos: Beyond the Call of Duty", cosine_sim_matrix)

## Recommendation function for Title

In [12]:
n_recommendation = 20
def get_recommendations(title, cosine_sim):

	title = games[games['title'] == title]['title'].values[0]
	print(title)

	# Get the index of the game that matches the name
	idx = indices[title]
	print(idx)

	# if there's 2 games or more with same name
	if type(idx) is pd.Series:
		return []
	
	similarity_array = cosine_sim[idx].toarray().flatten() #only if dense output is false
	# Get the pairwise similarity scores of all games with that game
	sim_scores = list(enumerate(similarity_array))

	# Sort the games based on the similarity scores
	print(sim_scores)
	sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

	# Get the scores of the most similar games
	# (not the first one because this games as a score of 1 (perfect score) similarity
    # with itself)
	sim_scores = sim_scores[1:n_recommendation + 1]

	# Get the games indices
	movie_indices = [i[0] for i in sim_scores]

	# Return the top most similar games
	return games['title'].iloc[movie_indices].tolist()

get_recommendations("Call of Duty®: Black Ops Cold War", cosine_sim_matrix)

Call of Duty®: Black Ops Cold War
2145
[(0, np.float64(0.0)), (1, np.float64(0.0)), (2, np.float64(0.0)), (3, np.float64(0.0)), (4, np.float64(0.0)), (5, np.float64(0.0)), (6, np.float64(0.0)), (7, np.float64(0.0)), (8, np.float64(0.0)), (9, np.float64(0.0)), (10, np.float64(0.0)), (11, np.float64(0.0)), (12, np.float64(0.0)), (13, np.float64(0.0)), (14, np.float64(0.0)), (15, np.float64(0.0)), (16, np.float64(0.0)), (17, np.float64(0.0)), (18, np.float64(0.0)), (19, np.float64(0.0)), (20, np.float64(0.0)), (21, np.float64(0.0)), (22, np.float64(0.0)), (23, np.float64(0.0)), (24, np.float64(0.0)), (25, np.float64(0.0)), (26, np.float64(0.0)), (27, np.float64(0.0)), (28, np.float64(0.0)), (29, np.float64(0.0)), (30, np.float64(0.0)), (31, np.float64(0.0)), (32, np.float64(0.0)), (33, np.float64(0.0)), (34, np.float64(0.0)), (35, np.float64(0.0)), (36, np.float64(0.0)), (37, np.float64(0.0)), (38, np.float64(0.0)), (39, np.float64(0.0)), (40, np.float64(0.0)), (41, np.float64(0.0)), (42,

['Combat Mission Cold War',
 'Duty in the Forest',
 'Cold House',
 'Hired Ops',
 'The Cold Forest',
 'RED OPS ARCADE',
 "Wizard's Duty",
 'Cold Way',
 'Cold Land',
 'Call of Duty®: Vanguard',
 'The Last War',
 "I'm on Observation Duty 5",
 "I'm on Observation Duty 4",
 'Cold Shell',
 'Cold Hill',
 'Cold Breath',
 'COLD DEPTH',
 'Black Cat',
 'Call of Duty®: Modern Warfare®',
 'Cold Verdict']

# Alternative recommendation By description
###### Currently only uses the final chunk to get recommendations not entire data cuz size too big 12gigs
###### Possible solution is to get just one random chunk of limited data to load chunk(Not entire dataset so no need of looping to process chunks) and get similarity_matrix dynamically but larger chunk can be time consuming. FYI Haven't tried this also probably won't work if title not in loaded chunk.
###### Or just use the following code, there are redundant process but it works
##### CAREFUL!! MAY USE RAM BEYOND CAPACITY AND START USING DISK dont use commented code unless you know what you are doing
###### Also the csv used here have already tokenized data, To tokenize new csv data refer to commented code Above that uses nltk, I forgor which one i used


In [6]:
games = reduce_memory(pd.read_csv('./games_preprocessed_with_tags_porterstemmer.csv'))
print(games.shape)
n_recommendation = 20
# Create a mapping from title to index
title_to_index = pd.Series(games.index, index=games['title']).to_dict() ##or just index it in recommendation function. Didn't do it cuz lazy

# Process data in chunks
chunk_size = 10000
chunks = pd.read_csv('./games_preprocessed_with_tags_porterstemmer.csv', chunksize=chunk_size)

count = TfidfVectorizer()
tfidf_matrices = []

for chunk in chunks:
    chunk.dropna(subset=['Tags'], inplace=True)
    tfidf_matrix_chunk = count.fit_transform(chunk['Tags'])
    tfidf_matrices.append(tfidf_matrix_chunk)

total_docs = sum(matrix.shape[0] for matrix in tfidf_matrices)
print(total_docs)
cosine_sim_matrix = np.zeros((total_docs, total_docs))
row_offset = 0

for matrix_chunk in tfidf_matrices:
    chunk_size = matrix_chunk.shape[0]
    cosine_sim_matrix[row_offset:row_offset + chunk_size, row_offset:row_offset + chunk_size] = cosine_similarity(matrix_chunk, matrix_chunk)
    row_offset += chunk_size

# Function to get recommendations
def get_recommendations(title, cosine_sim):
    if title not in title_to_index:
        return []

    idx = title_to_index[title]
    print(idx)
    print(cosine_sim.shape)
    # Ensure index is within bounds
    if idx >= cosine_sim.shape[0]:
        raise IndexError(f"Index {idx} is out of bounds for cosine similarity matrix with shape {cosine_sim.shape}.")

    # Get the pairwise similarity scores of all games with that game
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the games based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    print(sim_scores)
    # Get the scores of the most similar games
    sim_scores = sim_scores[1:n_recommendation + 1]

    # Get the games indices
    game_indices = [i[0] for i in sim_scores]
    print(game_indices)
    # Return the top most similar games
    return games['title'].iloc[game_indices].tolist()


# Example usage
# pickle.dump(cosine_sim_matrix,open('similarity_desc.pkl','wb'))
# np.savez_compressed('cosine_sim_matrix.npz', cosine_sim_matrix)
# recommendations = get_recommendations("Prince of Persia®: The Sands of Time", cosine_sim_matrix)
recommendations = get_recommendations("METAL SLUG 3", cosine_sim_matrix)
# recommendations = get_recommendations("Call of Duty®: Advanced Warfare - Gold Edition", cosine_sim_matrix)
print(recommendations)

(40484, 3)
40484
4
(40484, 40484)
[(4, np.float64(0.9999999999999996)), (400, np.float64(0.2831792372466566)), (29, np.float64(0.2730083740706562)), (556, np.float64(0.26837266271674254)), (480, np.float64(0.26828221139838465)), (49, np.float64(0.2632709375052442)), (758, np.float64(0.26037802924337683)), (922, np.float64(0.2416252831466136)), (582, np.float64(0.2330825823627601)), (1, np.float64(0.2328281043237895)), (691, np.float64(0.22598430754598736)), (280, np.float64(0.22549246959946276)), (945, np.float64(0.2243176857894409)), (723, np.float64(0.22206774852806194)), (771, np.float64(0.22016492749138752)), (613, np.float64(0.21790313653925933)), (27, np.float64(0.2118371409448837)), (778, np.float64(0.20815488394700754)), (756, np.float64(0.20417544459951267)), (143, np.float64(0.19877637775125948)), (987, np.float64(0.19655500291617248)), (253, np.float64(0.19374089609378498)), (576, np.float64(0.1895615460577812)), (167, np.float64(0.18888572827493677)), (937, np.float64(0.185

# Potentially better algo (Not Really)

In [2]:
# import pandas as pd
# import numpy as np

# def reduce_memory(df):
#     for col in df.columns:
#         if df[col].dtype == 'float64':
#             df[col] = df[col].astype('float32')
#         if df[col].dtype == 'int64':
#             df[col] = df[col].astype('int32')
#     return df

# # Load the games data
# games = reduce_memory(pd.read_csv('../games_preprocessed_with_tags_porterstemmer.csv'))
# n_recommendation = 20

# # Create a mapping from title to index
# title_to_index = pd.Series(games.index, index=games['title']).to_dict()

# # Load the cosine similarity matrix with memory mapping
# mmap_cosine_sim_matrix = np.load('cosine_sim_matrix.npz', mmap_mode='r')['arr_0']  # Use the appropriate key

# def get_recommendations(title, cosine_sim, chunk_size=1000):
#     if title not in title_to_index:
#         return []

#     idx = title_to_index[title]
    
#     # Ensure index is within bounds
#     if idx >= cosine_sim.shape[0]:
#         raise IndexError(f"Index {idx} is out of bounds for cosine similarity matrix with shape {cosine_sim.shape}.")

#     # Process in chunks to avoid high memory usage
#     sim_scores = []
#     for start in range(0, cosine_sim.shape[0], chunk_size):
#         end = min(start + chunk_size, cosine_sim.shape[0])
#         sim_scores_chunk = cosine_sim[idx, start:end].toarray().flatten()
#         sim_scores.extend(enumerate(sim_scores_chunk, start=start))

#     # Sort and filter as before
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:n_recommendation + 1]

#     game_indices = [i[0] for i in sim_scores]
#     return games['title'].iloc[game_indices].tolist()


# # Example usage
# recommendations = get_recommendations("METAL SLUG 3", mmap_cosine_sim_matrix)
# print(recommendations)


KeyboardInterrupt: 

In [26]:
description_df = pd.read_json('valid_games_metadata.json', encoding='utf-8')

# Extract the app_id and description columns
description_df = description_df[['app_id', 'description','tags']]
description_df.to_csv('games_metadata.csv')

In [ ]:
docs = ["Amidst the scorched sands of ancient Persia, there is a legend spun in an ancient tongue. It speaks of a time borne by blood and ruled by deceit. Drawn to the dark powers of a magic dagger, a young Prince is led to unleash a deadly evil upon a beautiful kingdom.",
        "Shapelab by Leopoly is a VR design application offering a variety of powerful polygon mesh-based sculpting tools, for both aspiring creatives and experienced 3D artists. Step into our virtual lab and release your creativity, make your own models, and perfect your ideas through digital sculpting!"]
cv = TfidfVectorizer()
word_count_vector = cv.fit_transform(docs)
tf = pd.DataFrame(word_count_vector.toarray(), columns=cv.get_feature_names_out())
print(tf)

cosine_sim_matrix = cosine_similarity(word_count_vector, word_count_vector)
print(cosine_sim_matrix)

sim_scores = list(enumerate(cosine_sim_matrix[0]))

# Sort the games based on the similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


In [ ]:
import pandas as pd
import ast

# Read the CSV file
df = pd.read_csv('new_games_data_withArryDesc.csv')

# Convert the strings back into lists
df['description'] = df['description'].apply(lambda x: ast.literal_eval(x))

# Display the DataFrame
df